In [20]:
import pandas as pd
import folium
from folium.plugins import HeatMap

# Read the CSV file into a DataFrame
data = pd.read_csv('/content/COVID-19.csv')

# Filter relevant columns for COVID-19 data
covid_data = data[['Date', 'CountryName', 'lat', 'lon', 'CumulativePositive']]

# Convert 'Date' column to datetime format
covid_data['Date'] = pd.to_datetime(covid_data['Date'])

# Ensure 'CumulativePositive' column is numeric
covid_data['CumulativePositive'] = pd.to_numeric(covid_data['CumulativePositive'], errors='coerce')

# Drop rows with NaN values in 'lat', 'lon', or 'CumulativePositive' columns
covid_data.dropna(subset=['lat', 'lon', 'CumulativePositive'], inplace=True)

# Group by 'CountryName' and sum the numeric columns
covid_data = covid_data.groupby('CountryName').agg({
    'Date': 'first',
    'lat': 'mean',
    'lon': 'mean',
    'CumulativePositive': 'sum'
}).reset_index()

# Calculate cases per capita
# Since population data is not available, we will not calculate cases per capita

# Print the highest and lowest risk countries
highest_risk_country = covid_data.groupby('CountryName')['CumulativePositive'].sum().idxmax()
lowest_risk_country = covid_data.groupby('CountryName')['CumulativePositive'].sum().idxmin()
print("Highest Risk Country:", highest_risk_country)
print("Lowest Risk Country:", lowest_risk_country)

# Create a folium map centered around the mean latitude and longitude
map = folium.Map(location=[covid_data['lat'].mean(), covid_data['lon'].mean()], zoom_start=3)

# Create a HeatMap layer
heatmap_data = covid_data[['lat', 'lon', 'CumulativePositive']].values.tolist()
folium.TileLayer('cartodbpositron').add_to(map)  # Optional: Add a basemap
HeatMap(heatmap_data).add_to(map)

# Save the map as an HTML file
map.save("covid_heatmap.html")

# Add instructions for high-risk and low-risk countries to the HTML file
high_risk_instructions = f"<h3>High-Risk Countries</h3><p>{highest_risk_country}: Travel to these countries is not recommended. Quarantine measures are advised for travelers returning from these countries.</p>"
low_risk_instructions = f"<h3>Low-Risk Countries</h3><p>{lowest_risk_country}: Travel to these countries is relatively safe, but adhere to preventive measures such as wearing masks and practicing social distancing.</p>"

print("High-risk instructions:", high_risk_instructions)
print("Low-risk instructions:", low_risk_instructions)

with open("covid_heatmap.html", "a") as file:
    file.write(high_risk_instructions)
    file.write(low_risk_instructions)


Highest Risk Country: United States of America
Lowest Risk Country: Saint-Barthélemy
High-risk instructions: <h3>High-Risk Countries</h3><p>United States of America: Travel to these countries is not recommended. Quarantine measures are advised for travelers returning from these countries.</p>
Low-risk instructions: <h3>Low-Risk Countries</h3><p>Saint-Barthélemy: Travel to these countries is relatively safe, but adhere to preventive measures such as wearing masks and practicing social distancing.</p>


<ipython-input-20-45222c9146f1>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  covid_data['Date'] = pd.to_datetime(covid_data['Date'])
<ipython-input-20-45222c9146f1>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  covid_data['CumulativePositive'] = pd.to_numeric(covid_data['CumulativePositive'], errors='coerce')
<ipython-input-20-45222c9146f1>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/